In [62]:
# Duplicate this template, write your scryfall query here and then run the notebook
QUERY = "t:creature sort:edhrec"
LIMIT = 500

In [63]:
import mtgstats.scryfall.api as api
import mtgstats.cards as c
from mtgstats.analysis import cmc_analysis, color_analysis, subtype_analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [64]:
query = api.search(QUERY, limit=LIMIT)
cards = c.parse_cards(query)

In [65]:
# Go for the Throat - Non-artifact
# Doomblade - non-black
# Malicious Afflitcion - non-black
# Victim of Night - non-Vampire, non-Werewolf, non-Zombie
# Vendetta - non-black, can't regenerate, loses life equal to toughness
# Terror - Non-black
# Ultimate Price - Monocolored
# Devour in shadow - Can't regenerate, loses life equal to toughness
# Power Word Kill - non-Angel, non-Demon, non-Devil, non-Dragon

In [81]:
KEYWORDS = ["Shroud", "Hexproof", "Indestructible"]

def has_any_keyword(keywords, card):
    return any( [kw in card.get('keywords', []) for kw in keywords])

f = lambda x: not has_any_keyword(KEYWORDS, x)

In [82]:
destructible = list(filter(f, cards))
c = color_analysis(destructible)
cmc, cum_cmc = cmc_analysis(destructible)
types = subtype_analysis(destructible)
data = pd.DataFrame(destructible)

Nissa, Vastwood Seer
Glasspool Mimic
Docent of Perfection
Birgi, God of Storytelling
Liliana, Heretical Healer


In [83]:
# Número de criaturas não pretas: Doomblade, Malicious Affliction, Vendetta
non_black_creatures = len(data) - c['B']
print(non_black_creatures)

378


In [84]:
artifact_creatures =  len(data[data['type_line'].str.contains("Artifact")])
# Número de criaturas artefatos
print(len(data) - artifact_creatures)
# Terror
print(non_black_creatures - artifact_creatures)

439
339


In [85]:
# Numero de criaturas monocoloridas: Ultimate price
len(data[data.colors.str.len() == 1])

352

In [86]:
# Victim of Night
t = ['Zombie', 'Werewolf', 'Vampire']
len(data) - sum([types.get(x, 0) for x in t])

440

In [87]:
# Power Word Kill
t = ['Angel', 'Demon', 'Devil', 'Dragon']
len(data) - sum([types.get(x, 0) for x in t])

441

In [88]:
def tough_helper(x):
    t = x.get('toughness', 0)
    # Cartas com * na resistência vão ficar marcadas com 0
    return int(t) if t != '*' else 0

In [89]:
# Distribuição das resistências para Vendetta (criaturas não pretas)
t = list(
    map(tough_helper, 
        filter(lambda x: 'B' not in x.get('colors', []),
               destructible)))
from collections import Counter
counter = Counter(t)
s = sorted(counter.items(), key=lambda x:x[0])

_, t = zip(*s)
# Soma cumulativa <resitencia>: <soma>
dict(zip(_,np.cumsum(t)))

{0: 25,
 1: 113,
 2: 204,
 3: 250,
 4: 300,
 5: 329,
 6: 358,
 7: 366,
 8: 367,
 9: 372,
 10: 373,
 11: 374,
 12: 377,
 13: 378}

In [90]:
# Distribuição das resistências para Malicious Afliction
t = list(
    map(tough_helper,destructible))
from collections import Counter
counter = Counter(t)
s = sorted(counter.items(), key=lambda x:x[0])

_, t = zip(*s)
# Soma cumulativa <resitencia>: <soma>
dict(zip(_,np.cumsum(t)))

{0: 26,
 1: 131,
 2: 248,
 3: 313,
 4: 384,
 5: 419,
 6: 456,
 7: 464,
 8: 467,
 9: 472,
 10: 473,
 11: 474,
 12: 477,
 13: 478}